# Deep Learning with PyTorch
---
## chapter 2 Tensor

In [1]:
import torch

In [2]:
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [3]:
a[1]

tensor(1.)

In [4]:
float(a[1])

1.0

In [5]:
a[2]=2.0
a

tensor([1., 1., 2.])

从表面来看，tensor结构似乎和list结构没有太多的不同  
但是实际上在底层，已经完全不同了  
- list 中各元素实际上是相互独立地存放在内存中的
- tensor，或者numpy中的array，是存储在一段连续内存中的，底层是C

---
二维tensor

In [6]:
points = torch.tensor([[1.0, 4.0], [2.0, 1.0], [3.0, 5.0]])
points

tensor([[1., 4.],
        [2., 1.],
        [3., 5.]])

对二维tensor的索引也就是在索引位置依次写其所在的索引值，比如第一个tensor的第二个值，那就是0，1

In [7]:
print(points.shape)
print(points[0, 1])
print(points[0])

torch.Size([3, 2])
tensor(4.)
tensor([1., 4.])


In [8]:
zeroTensor = torch.zeros(3, 2)
zeroTensor

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

**注意：以上输出结果已经是另一个tensor了**

---
#### storage

In [9]:
points.storage()

 1.0
 4.0
 2.0
 1.0
 3.0
 5.0
[torch.FloatStorage of size 6]

虽然tensor是三行两列的，但是storage或者叫存储是一个大小为6的连续数组结构  
storage 的结构一般是一维的

In [10]:
points_storage = points.storage()
points_storage[0]

1.0

如果修改了storage中的值，那么也会修改引用它的tensor里面的值

In [11]:
points_storage[0] = 2.0
points

tensor([[2., 4.],
        [2., 1.],
        [3., 5.]])

所以要避免直接使用storage的实例，但是理解一个tensor和storage背后的联系对理解程序的开销是很有帮助的

In [12]:
second_point = points[1]
second_point.storage_offset()

2

In [13]:
second_point.size()

torch.Size([2])

In [14]:
second_point.shape

torch.Size([2])

In [15]:
points.stride()

(2, 1)

subtensor  
修改subtensor同样会改变原始的tensor  
在这里的subtensor其实引用的storage和原始的tensor是一样的

In [16]:
second_point.stride()

(1,)

In [17]:
second_point[0]=10.0
points

tensor([[ 2.,  4.],
        [10.,  1.],
        [ 3.,  5.]])

In [18]:
points = torch.tensor([[1.0, 4.0], [2.0, 1.0], [3.0, 5.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[1., 4.],
        [2., 1.],
        [3., 5.]])

In [19]:
points_t = points.t()
points_t

tensor([[1., 2., 3.],
        [4., 1., 5.]])

In [20]:
id(points.storage()) == id(points_t.storage())  # 比较两个storage的是不是同一个

True

上述比较说明，storage是相同的，他们的区别仅限于stride和shape

In [21]:
print(points.stride())
print(points_t.stride())

(2, 1)
(1, 2)


transpose可以进行转置，也可以交换两个指定的维数 - 对应书上35/141的位置

In [22]:
some_tensor = torch.ones(3, 4, 5)
some_tensor_t = some_tensor.transpose(0, 2)
print(some_tensor.shape)
print(some_tensor_t.shape)

torch.Size([3, 4, 5])
torch.Size([5, 4, 3])


In [23]:
some_tensor.stride()

(20, 5, 1)

stride是tensor在矩阵形式和一维形式之间转换的步长，是一个比较重要的量

In [24]:
some_tensor_t.stride()

(1, 5, 20)

 points 是 contiguous 的，连续的？  
 但是它的转置不是  
 contiguous 的定义是

In [25]:
print(points.is_contiguous())
print(points_t.is_contiguous())

True
False


使用contiguous 方法可以把一个 noncontiguous的张量tensor编程时contiguous的  
会改变stride

In [26]:
some_tensor_t_cont = some_tensor.contiguous()
some_tensor_t_cont.stride()

(20, 5, 1)

## 索引
**注意：pytorch tensor 的索引第一个量是行（rows），第二个量是列（columns）**  
pytorch还有一种 advanced indexing，似乎直接是有过接触的，但是可能还没有怎么用过

In [27]:
print(points[1:])
print(points[1:, :1])

tensor([[2., 1.],
        [3., 5.]])
tensor([[2.],
        [3.]])


## 和Numpy的互通性
pytorch tensor 可以转化为 NumPy array

In [28]:
points_np = points.numpy()
points_np

array([[1., 4.],
       [2., 1.],
       [3., 5.]], dtype=float32)

In [29]:
points_re = torch.from_numpy(points_np)
points_re

tensor([[1., 4.],
        [2., 1.],
        [3., 5.]])

## 保存
可以将数据保存到文件中

In [30]:
torch.save(points, './data/onepoints.t')

In [31]:
with open('./data/onepoints.t', 'wb') as f:
    torch.save(some_tensor, f)

In [32]:
points_f = torch.load('./data/onepoints.t')
points_f

tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]])

In [33]:
with open('./data/onepoints.t', 'rb') as f:
    points_f = torch.load(f)

### 关于 HDF5 format
该格式允许以 NumPy array 的形式进行保存

In [34]:
import h5py

In [35]:
f = h5py.File('./data/onepoints.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy())
f.close()

`coords` 是一个HDF5文件的关键词，原文说的是：Here, 'coords' is a key into the HDF5 file，其用法可以看下面，感觉是起了一个名字

In [36]:
f = h5py.File('./data/onepoints.hdf5', 'r')
dset = f['coords']
last_points = dset[1:]

上面这种调用方式的好处是，数据不会在文件打开时进行调用，只在使用这些数据（last_points）时才会被加载，而且会返回一个类似NumPy数组的、只包含你所调用的内容（比如上面只有两行），但是作为numpy array的性质是完全一致的

使用 `from_numpy` 可以将上面的数据重新转化为tensor

In [37]:
last_points = torch.from_numpy(dset[1:])
f.close()

### tensor在GPU上

In [38]:
points_gpu = torch.tensor([[1.0, 4.0], [2.0, 1.0], [3.0, 4.0]], device='cuda')

也可以将tensor在CPU和GPU之间进行改变

In [39]:
points_gpu = points.to(device='cuda')

## 部分 API

In [40]:
a = torch.ones(3,2)
a.zero_()

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])